In [1]:
import os
import re
from glob import glob
import pandas as pd

In [2]:
read_csv = lambda file_path: pd.read_csv(file_path, encoding="cp949")

In [3]:
def get_file_date(file_path):
    file_name = os.path.basename(file_path)
    date = re.findall(r"\d{8}|\d{4}", file_name)[-1]
    if len(date) == 4:
        return date + "1231"
    elif len(date) == 8:
        return date
    else:
        raise ValueError

In [4]:
def format_df(df):
    formatted_df = df.melt(
        id_vars=["범죄대분류", "범죄중분류"],
        var_name="지역",
        value_name="Count",
    )
    return formatted_df


def append_column(df, column_name, column_value):
    df[column_name] = column_value
    return df

append_date = lambda df, date: append_column(df, "date", date)
preproc_data = lambda df, date: append_date(format_df(df), date)

In [5]:
main_cols = [
    "범죄대분류", "범죄중분류",
    
    "서울", "부산", "대구", "인천", "광주", "대전", "울산", "세종",
    
    "기타도시", "도시이외",
    
    "경기 고양", "경기 과천", "경기 광명", "경기 광주", "경기 구리", "경기 군포",
    "경기 김포", "경기 남양주", "경기 동두천", "경기 부천", "경기 성남", "경기 수원",
    "경기 시흥", "경기 안산", "경기 안성", "경기 안양", "경기 양주", "경기 여주",
    "경기 오산", "경기 용인", "경기 의왕", "경기 의정부", "경기 이천", "경기 파주",
    "경기 평택", "경기 포천", "경기 하남", "경기 화성",
    
    "강원 강릉", "강원 동해", "강원 삼척", "강원 속초", "강원 원주", "강원 춘천", "강원 태백",
    
    "충북 제천", "충북 청주", "충북 충주",
    
    "충남 계룡", "충남 공주", "충남 논산", "충남 당진", "충남 보령", "충남 서산",
    "충남 아산", "충남 천안",
    
    "전북 군산", "전북 김제", "전북 남원", "전북 익산", "전북 전주", "전북 정읍",
    
    "전남 광양", "전남 나주", "전남 목포", "전남 순천", "전남 여수",
    
    "경북 경산", "경북 경주", "경북 구미", "경북 김천", "경북 문경", "경북 상주",
    "경북 안동", "경북 영주", "경북 영천", "경북 포항",
    
    "경남 거제", "경남 김해", "경남 밀양", "경남 사천", "경남 양산", "경남 진주",
    "경남 창원", "경남 통영",    
    
    "제주 서귀포", "제주 제주",
]


_mapper_마산 = lambda x : '경남 마산' if x == '마산' else  x

def add_base(main_cols, col):
    if col in main_cols:
        return col
    
    exist_cols = [main_col for main_col in main_cols if main_col.endswith(col)]
    
    if len(exist_cols) == 1:
        return exist_cols[0]
    else:
        return col

_add_base = lambda col: add_base(main_cols, col)

do_strip = lambda col: col.strip()

preproc_columns = lambda cols: [_mapper_마산(_add_base(do_strip(col))) for col in cols]

In [6]:
file_paths = glob("./data/raw_data/*.csv")
file_paths

['./data/raw_data/경찰청_범죄발생지_2012.csv',
 './data/raw_data/경찰청_범죄발생지_2013.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20191231.csv',
 './data/raw_data/경찰청_범죄발생지_2014.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20181231.csv',
 './data/raw_data/경찰청_범죄발생지_2017.csv',
 './data/raw_data/경찰청_범죄발생지_2016.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20221231.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20211231.csv',
 './data/raw_data/경찰청_범죄발생지_20151231.csv',
 './data/raw_data/경찰청_범죄 발생 지역별 통계_20201231.csv']

In [7]:
df_list = list()
for file_path in file_paths:
    df = read_csv(file_path)
    df.columns = preproc_columns(df)
    date = get_file_date(file_path)
    preproc_data(df, date).to_csv(f"./data/preproc_data/crime_{date[:4]}.csv")